In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pwd

'/content'

In [9]:
cd /content/drive/MyDrive/GomokuAI

/content/drive/MyDrive/GomokuAI


# Git

In [ ]:
!git fetch

In [ ]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), 446 bytes | 0 bytes/s, done.
From https://github.com/onetwothr1/GomokuAI
   64cc183..10982e5  main       -> origin/main
Updating 64cc183..10982e5
Fast-forward
 compare_performance.py | 10 +++++++++-
 1 file changed, 9 insertions(+), 1 deletion(-)


In [ ]:
!git add .

In [ ]:
!git config --global user.email kimjh1101@snu.ac.kr 
!git config --global user.name onetwotr1

In [ ]:
!git commit -m "concentrate on defense"

[main a033bea] alphazero 2250
 14 files changed, 22 insertions(+), 12 deletions(-)
 rewrite __pycache__/human_v_bot.cpython-39.pyc (99%)
 delete mode 100644 models/alphazero 1250 111 loss.png
 create mode 100644 models/alphazero 1250 113 loss.png
 create mode 100644 models/alphazero 1750 101 loss.png
 rename models/{alphazero 1250 103.pt => alphazero 1750.pt} (100%)
 create mode 100644 models/alphazero 2000 110 loss.png
 create mode 100644 models/alphazero 2000.pt
 create mode 100644 models/alphazero 2250.pt
 rewrite project.ipynb (88%)


In [ ]:
!git push origin main

Enumerating objects: 27, done.


# Train

In [ ]:
import importlib
import train, utils
importlib.reload(train)
importlib.reload(utils)

<module 'utils' from '/content/drive/MyDrive/GomokuAI/utils.py'>

alphazero 0 -> 750번 자체 대국 12시간

alphazero 750 -> 500번 자체 대국 8시간

alphazero 1250 -> 500번 자체 대국 20시간 (rollout 400)

alphazero 1750

In [ ]:
!python train.py -m "models/alphazero 2000.pt" -save "models/l2 0.001" -d "experience/alphazero 2000 self-play 250x8 new.pickle" -n 1000 -lr 0.001 --lr-decay 0.95 --early-stop 10

num data: 43168
num mini-batch: 43
--------------------------------
Epoch 1
43it [00:07,  5.67it/s]
average loss 1.5528 (policy 1.0976,  value 0.4552)
--------------------------------
Epoch 2
43it [00:00, 70.16it/s]
average loss 1.3730 (policy 0.9960,  value 0.3770)
--------------------------------
Epoch 3
43it [00:00, 64.78it/s]
average loss 1.3201 (policy 0.9618,  value 0.3583)
--------------------------------
Epoch 4
43it [00:00, 63.42it/s]
average loss 1.2863 (policy 0.9407,  value 0.3456)
--------------------------------
Epoch 5
43it [00:00, 61.80it/s]
average loss 1.2626 (policy 0.9267,  value 0.3360)
--------------------------------
Epoch 6
43it [00:00, 63.44it/s]
average loss 1.2368 (policy 0.9123,  value 0.3245)
--------------------------------
Epoch 7
43it [00:00, 61.69it/s]
average loss 1.2168 (policy 0.9044,  value 0.3123)
--------------------------------
Epoch 8
43it [00:00, 63.38it/s]
average loss 1.1961 (policy 0.8977,  value 0.2984)
--------------------------------
Epoc

alphazero 0 80.pt를 선택! -> alphazero 750
750 vs 0 => 9:1

c=1.0 vs c=2.0 => 12:2 (p-value 0.012)

c=1.0 vs c=1.5 => 10:0

weight2 epoch 109 낙찰 -> alphazero 1250.pt


alphazero 1250 vs alphazero 750 => 20 : 0

alphazero 1250
rollout 400 vs 100 => 10:0

1750 : 1250 => 10:0  (아래의 1750-1) 을 사용)

alphazero 1750 후보

1) non weighted loss, no l2 constraint. epoch 60   (overfitting 되기 전이라 판단한 지점에서 끊음)

2) non weighted loss, l2 constraint (0.001) epoch 102  (맨 끝 epoch)

왠진 모르겠는데 l2 사용한 게 조금 더 빠르게 loss 떨어짐. policy, value 모두.



후보 1), 2) 비교 결과 2)가 3:17로 압승. value function overfitting이 우려됐으나 조금도 우려하지 않아도 되는 듯함

alphazero 2000 100 vs alphazero 2000 => 9 : 1
p-value 0.021484

# 성능 비교

In [ ]:
import importlib
import compare_performance, bot_v_bot, human_v_bot, utils
importlib.reload(compare_performance)
importlib.reload(bot_v_bot)
importlib.reload(human_v_bot)
importlib.reload(utils)

<module 'utils' from '/content/drive/MyDrive/GomokuAI/utils.py'>

In [ ]:
import importlib
import agent.alphazero_agent
importlib.reload(agent.alphazero_agent)

<module 'agent.alphazero_agent' from '/content/drive/MyDrive/GomokuAI/agent/alphazero_agent.py'>

In [ ]:
!python bot_v_bot.py

In [ ]:
!python compare_performance.py -m1 "models/alphazero 2250.pt" -m2 "models/alphazero 2000 91 new.pt" -n 20 --verbose 4 --use-model-name

In [ ]:
!python compare_performance.py -m1 "models/l2 0.001/alphazero 2000 100.pt" -m2 "models/alphazero 2000.pt" -n 10 --verbose 4 --use-model-name

# self play

In [ ]:
import importlib
import bot_v_bot, utils, agent.alphazero_agent, compare_performance, self_play, board
# import net.alphazero_net
# importlib.reload(bot_v_bot)
# importlib.reload(compare_performance)
importlib.reload(utils)
importlib.reload(self_play)
importlib.reload(board)
importlib.reload(agent.alphazero_agent)
# importlib.reload(net.alphazero_net)

<module 'agent.alphazero_agent' from '/content/drive/MyDrive/GomokuAI/agent/alphazero_agent.py'>

In [ ]:
!python self_play.py -n 1 -m "models/alphazero 2000 91 new.pt" --file-num 2 -rollout 400 -c 2 --noise-intensity 0.25 --alpha 0.1 --verbose 4

In [ ]:
for i in range(25, 100):
  print()
  print(i, 'th simulation')
  !python self_play.py -n 3 -m "models/alphazero 2000 91 new.pt" --file-num {i} -rollout 400 -c 2.5 --noise-intensity 0.25 --alpha 0.1 --verbose 4

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
D0     visit   5  p 0.004  v  0.40  exp_v -0.87
A7     visit   5  p 0.046  v  0.59  exp_v -0.91
A8     visit   5  p 0.007  v  0.33  exp_v -0.61
B0     visit   4  p 0.001  v  0.43  exp_v -0.85
C1     visit   4  p 0.008  v  0.40  exp_v -0.56
G7     visit   4  p 0.012  v  0.59  exp_v -0.65
J7     visit   4  p 0.004  v  0.57  exp_v -0.54
A1     visit   3  p 0.016  v  0.35  exp_v -0.78
lose expected
----------------------------
Player.white F8
 8     ●●●○●    
 7     ○○  ○      
 6 ○  ●○●○      
 5 ○●●○●○○    
 4     ●○○●●○  
 3     ●●○●●●  
 2     ○●○○  ●  
 1         ○●○●○
 0 ●  ○  ●○○●  
   A B C D E F G H J
average depth: 3.16, max depth: 6
A8     visit 275  p 0.025  v -0.05  exp_v  0.56
B8     visit  14  p 0.169  v -0.17  exp_v -0.06
D0     visit  13  p 0.019  v -0.13  exp_v  0.21
A1     visit  13  p 0.020  v  0.14  exp_v  0.11
B7     visit  11  p 0.091  v -0.08  exp_v  0.10
J8     visit  11  p 0.030  v -0.01  exp_v  0.09
H5     visit   9  p 0.030  v

# 데이터 합치기

In [ ]:
import os
_list = os.listdir('experience')

In [ ]:
from pprint import pprint
files = [i for i in _list if '2000' in i and '.pickle' in i]
print(len(files))
pprint(files)

In [ ]:
import importlib
import experience
importlib.reload(experience)

<module 'experience' from '/content/drive/MyDrive/GomokuAI/experience.py'>

In [ ]:
from experience import *

combine_saved_experiences(['experience/' + f for f in files], "experience/alphazero 2000 self-play 250.pickle")

experience/alphazero 2000 self-play 5.pickle


/content/drive/MyDrive/GomokuAI/experience.py:105: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  self.mcts_probs = torch.tensor(self.mcts_probs)
/content/drive/MyDrive/GomokuAI/experience.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rewards_list.append(torch.tensor(collector.rewards))


113
experience/alphazero 2000 self-play 5 1.pickle
108
experience/alphazero 2000 self-play 5 2.pickle
97
experience/alphazero 2000 self-play 5 3.pickle
133
experience/alphazero 2000 self-play 5 4.pickle
103
experience/alphazero 2000 self-play 5 5.pickle
117
experience/alphazero 2000 self-play 5 6.pickle
110
experience/alphazero 2000 self-play 5 7.pickle
106
experience/alphazero 2000 self-play 5 8.pickle
106
experience/alphazero 2000 self-play 5 9.pickle
93
experience/alphazero 2000 self-play 5 10.pickle
116
experience/alphazero 2000 self-play 5 11.pickle
118
experience/alphazero 2000 self-play 5 12.pickle
102
experience/alphazero 2000 self-play 5 13.pickle
112
experience/alphazero 2000 self-play 5 14.pickle
84
experience/alphazero 2000 self-play 5 15.pickle
115
experience/alphazero 2000 self-play 5 16.pickle
84
experience/alphazero 2000 self-play 5 17.pickle
81
experience/alphazero 2000 self-play 5 18.pickle
108
experience/alphazero 2000 self-play 5 19.pickle
93
experience/alphazero 20

In [ ]:
def seconds_to_hms(seconds):
    hours = seconds // 3600
    remaining_seconds = seconds % 3600
    minutes = remaining_seconds // 60
    remaining_seconds = remaining_seconds % 60

    return hours, minutes, remaining_seconds

In [ ]:
txt_files = [i for i in _list if '2000' in i and '.pickle' in i]
for f in txt_files:
    'experience/' +f

In [ ]:
from experience import *

combined = ExperienceCollector()
combined.load_experience("experience/alphazero 2000 self-play 250.pickle")
augmented = augmentation(combined, 9)
print(len(augmented))
print(len(augmented.mcts_probs))
print(len(augmented.rewards))

43168
43168
43168


In [ ]:
augmented.save_experience("experience/alphazero 2000 self-play 250x8.pickle")

# Play

In [ ]:
import importlib
import human_v_bot
importlib.reload(human_v_bot)

<module 'human_v_bot' from '/content/drive/MyDrive/GomokuAI/human_v_bot.py'>

In [ ]:
!python human_v_bot.py --verbose 4